In [ ]:
import pyodbc
import pandas as pd

cnxn = pyodbc.connect('DRIVER={Cloudera ODBC Driver for Impala};HOST=HIDDEN;PORT=25003;AuthMech=1;UseSASL=1;DATABASE=phd;',
                      autocommit=True)
cursor = cnxn.cursor()

cursor.execute("Show tables;")
row = cursor.fetchone()
while row:
    print(row[0])
    row = cursor.fetchone()


In [ ]:
# test query 
query = ("SELECT a1.STD_CHG_CODE as id1, a2.STD_CHG_CODE as id2, COUNT(1) as weight FROM " +
         "(SELECT b.PAT_KEY, b.SERV_DAY, b.STD_CHG_CODE, b.STD_QTY FROM phd_20163_patbill b " +
         "ORDER BY b.PAT_KEY, b.SERV_DAY LIMIT 1000) a1 " +
         "JOIN (SELECT b.PAT_KEY, b.SERV_DAY, b.STD_CHG_CODE, b.STD_QTY FROM phd_20163_patbill b " +
         "ORDER BY b.PAT_KEY, b.SERV_DAY LIMIT 1000) a2 " +
         "ON a1.PAT_KEY = a2.PAT_KEY AND a1.SERV_DAY = a2.SERV_DAY AND a1.STD_CHG_CODE < a2.STD_CHG_CODE " +
         "GROUP BY a1.STD_CHG_CODE, a2.STD_CHG_CODE ORDER BY COUNT(1) DESC")

df = pd.read_sql(query, cnxn)
print(df.shape)

In [ ]:
rel_list = []
df = None
for i in range(10, 19):
    for j in range(1, 5):
        if i == 18 and j == 4:
            continue
        
        print(i, j)
        query = ("SELECT a1.STD_CHG_CODE as id1, a2.STD_CHG_CODE as id2, COUNT(1) as weight FROM " +
                 "(SELECT b.PAT_KEY, b.SERV_DAY, b.STD_CHG_CODE, b.STD_QTY FROM phd_20" + str(i) + str(j) + "_patbill b " +
                 "ORDER BY b.PAT_KEY, b.SERV_DAY) a1 " +
                 "JOIN (SELECT b.PAT_KEY, b.SERV_DAY, b.STD_CHG_CODE, b.STD_QTY FROM phd_20" + str(i) + str(j) + "_patbill b " +
                 "ORDER BY b.PAT_KEY, b.SERV_DAY) a2 " +
                 "ON a1.PAT_KEY = a2.PAT_KEY AND a1.SERV_DAY = a2.SERV_DAY AND a1.STD_CHG_CODE < a2.STD_CHG_CODE " +
                 "GROUP BY a1.STD_CHG_CODE, a2.STD_CHG_CODE ORDER BY COUNT(1) DESC")
        
        res_df = pd.read_sql(query, cnxn)
        res_df.set_index(['id1', 'id2'], inplace=True)
        res_df['weight'].astype('int64', inplace=True)
        
        if df is None:
            df = res_df
        else:
            df = df.add(res_df, fill_value=0).astype('int64')
            
        df.sort_values(['id1', 'id2'], inplace=True)
        print(df.shape)
        print(df.head())

In [ ]:
print(df.shape)

In [ ]:
%matplotlib inline
print(df.head(100))
print(df['weight'].dtype)
df['weight'].hist()

In [ ]:
import pandas as pd
df = pd.read_csv('./features/rels.csv', header=0, index_col=['id1', 'id2'])
print(df.shape)
print(df.columns)
print(df.head())

In [ ]:
print(df['weight'].sum())

In [ ]:
df_10 = df[df['weight'] >= 10]
print(df_10.shape)
df_10.to_csv('./features/rels_10.csv')